In [1]:
pwd()

'/home/bio/source/notebooks'

In [9]:
cd /data/nijhawanlab/tiana

/data/nijhawanlab/tiana


In [8]:
ls /data/nijhawanlab/tiana/

30-439930955_GENEWIZ_Data_Report_201116181014779.html
CRISPResso_on_PCR1-read-1.fasta_PCR1-read-2.fasta/
CRISPResso_on_PCR1-read-1_PCR1-read-2/
CRISPResso_on_PCR1-read-1_PCR1-read-2.html
CRISPResso_on_PCR2-read-1_PCR2-read-2/
CRISPResso_on_PCR2-read-1_PCR2-read-2.html
CRISPResso_on_PCR2-read-1_PCR2-read-2.zip
CRISPResso_on_PCR3-read-1_PCR3-read-2/
CRISPResso_on_PCR3-read-1_PCR3-read-2.html
CRISPResso_on_TN-LSS-KO_R2_001/
CRISPResso_on_TN-LSS-KO_R2_001.html
GENEWIZ_Data_Download_Guide.pdf
P1/
P1_R1.fasta
P1_R1.fastq
P2/
PCR1
PCR1-read-1.fastq
PCR1-read-2.fastq
PCR2
PCR2-read-1.fastq
PCR2-read-2.fastq
PCR3
PCR3-read-1.fastq
PCR3-read-2.fastq
PCR4
PCR4-read-1.fastq
PCR4-read-2.fastq
PCR4.1.reads
PCR4.counts
PCR5
PCR5-read-1.fastq
PCR5-read-2.fastq
PCR6
PCR6-read-1.fastq
PCR6-read-2.fastq
PCR7
PCR7-read-1.fastq
PCR7-read-2.fastq
PCR8
PCR8-read-1.fastq
PCR8-read-2.fastq
R1.fixed.fasta
R1.fixed.fastq
R2.fixed.fasta
R2.fixed.fastq
TN-LSS-KO_R1_001.fasta
TN-LSS-KO_R1_001.fastq
TN-LSS-KO_R1_001

In [10]:
from Bio import Seq, SeqIO
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import csv
import itertools
import re
import xlwt, openpyxl

In [12]:
%cd /data/nijhawanlab/tiana/
R1_file = "TN-LSS-KO_R1_001.fastq.gz"
R2_file = "TN-LSS-KO_R2_001.fastq.gz"
primer_file = "primers.tsv"

/data/nijhawanlab/tiana


In [169]:
primers=pd.read_table(primer_file, header=None, 
              names=['Name','Forward','Reverse'],
              converters={'Name':lambda s: s.upper(),'Forward':lambda s: s.upper(),'Reverse':lambda s: s.upper()}
             )
primers

,Name,Forward,Reverse
0,PCR1,ATGTTCTTCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
1,PCR2,ATGTCTCCCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
2,PCR3,GCACTCAACTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
3,PCR4,TAGTAGCCCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
4,PCR5,CGTGTCAACTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
5,PCR6,GCGTTCTTCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
6,PCR7,GCCAAGCCCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG
7,PCR8,GCACCTCCCTGGGCGGAGTCTAAGGAAG,GGATGGAATGGTGCTTTCAG


In [176]:
R1 = gzip.open(R1_file, "rt")
R2 = gzip.open(R2_file, "rt")
reads1 = SeqIO.parse(R1, "fastq")
reads2 = SeqIO.parse(R2, "fastq")
maxReads = 1000
readNumber = 1
reads_list = []
for (read1, read2) in zip(reads1, reads2):
    # print(read1)
    R1_sequence = str(read1.seq)
    R2_sequence = str(read2.seq)
    R1_ID = str(read1.id)
    R2_ID = str(read2.id)
    
    primer_matches = primerMatches(R1_sequence, R2_sequence, primers)
    if primer_matches == ():
        primer_matches = primerMatches(R2_sequence, R1_sequence, primers)
        direction = 'R'
    else:
        direction = 'F'
    
    reads_list.append({
        'R1_seq':R1_sequence,
        'R2_seq':R2_sequence,
        'R1_ID':R1_ID,
        'R2_ID':R2_ID,
        'primers':primer_matches,
        'direction':direction
    })
    readNumber+=1
    if readNumber > maxReads:
        break
    
reads_table = pd.DataFrame(reads_list)

In [177]:
reads_table

,R1_seq,R2_seq,R1_ID,R2_ID,primers,direction
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,"(PCR5,)",R
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,"(PCR5,)",F
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,"(PCR5,)",R
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,"(PCR2,)",R
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,"(PCR6,)",F
...,...,...,...,...,...,...
995,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,"(PCR2,)",R
996,ACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTC...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCGGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,(),R
997,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,"(PCR2,)",R
998,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,"(PCR7,)",F


In [178]:
reads_table.loc[lambda df: df['primers'] != (), :]

,R1_seq,R2_seq,R1_ID,R2_ID,primers,direction
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,"(PCR5,)",R
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,"(PCR5,)",F
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,"(PCR5,)",R
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,"(PCR2,)",R
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,"(PCR6,)",F
...,...,...,...,...,...,...
994,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,"(PCR7,)",F
995,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,"(PCR2,)",R
997,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,"(PCR2,)",R
998,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,"(PCR7,)",F


In [149]:
reads_table['sanity'] = (
    (reads_table['R1_ID'] == reads_table['R2_ID'])
)

In [175]:
def forwardPrimerMatches(seq, primers):
    matches = []
    # print('forwardPrimerMatches: ', seq,'\n')
    for i in range(len(primers)):
        # print(i, ': ', primers.iloc[i]['Forward'])
        if seq.startswith(primers.iloc[i]['Forward']):
            matches.append(primers.iloc[i]['Name'])
    # print('\n')
    return matches

def reversePrimerMatches(seq, primers):
    matches = []
    # print('reversePrimerMatches: ', seq,'\n')
    for i in range(len(primers)):
        # print(i, ': ', primers.iloc[i]['Reverse'])
        if seq.startswith(primers.iloc[i]['Reverse']):
            matches.append(primers.iloc[i]['Name'])
    # print('\n')
    return matches

def primerMatches(fwd_seq, rev_seq, primers):
    return tuple(p for p in forwardPrimerMatches(fwd_seq, primers) if p in reversePrimerMatches(rev_seq, primers))

In [150]:
reads_table['primers'] = reads_table.apply(
    lambda r: primerMatches(r['R1_seq'], r['R2_seq'], primers), 
    axis=1)

In [106]:
reads_table

,R1_seq,R2_seq,R1_ID,R2_ID,sanity,primers
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,True,()
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,True,"(PCR5,)"
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,True,()
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,True,()
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,True,"(PCR6,)"
...,...,...,...,...,...,...
995,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,True,()
996,ACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTC...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCGGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,True,()
997,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,True,()
998,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,True,"(PCR7,)"


In [107]:
# select all the reads with at least one primer match:
reads_table.loc[lambda df: df['primers'] != (), :]

,R1_seq,R2_seq,R1_ID,R2_ID,sanity,primers
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,True,"(PCR5,)"
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,True,"(PCR6,)"
5,GCACTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1280:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1280:...,True,"(PCR3,)"
6,ATGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14805...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14805...,True,"(PCR1,)"
7,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:5773:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:5773:...,True,"(PCR5,)"
...,...,...,...,...,...,...
988,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:24496...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:24496...,True,"(PCR5,)"
991,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:30481...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:30481...,True,"(PCR2,)"
992,ATGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:24243...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:24243...,True,"(PCR1,)"
994,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,True,"(PCR7,)"


In [179]:
reads_table.loc[lambda df: df['primers'] == ('PCR5',), :]

,R1_seq,R2_seq,R1_ID,R2_ID,primers,direction
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,"(PCR5,)",R
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,"(PCR5,)",F
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,"(PCR5,)",R
7,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:5773:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:5773:...,"(PCR5,)",F
11,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:16206...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:16206...,"(PCR5,)",F
...,...,...,...,...,...,...
941,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:31195...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:31195...,"(PCR5,)",F
962,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:12879...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:12879...,"(PCR5,)",F
975,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26169...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26169...,"(PCR5,)",R
982,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1325:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1325:...,"(PCR5,)",F


In [180]:
reads_table['N_primers'] = reads_table.apply(
    lambda r: len(r['primers']), 
    axis=1)
reads_table['sanity'] = (
    (reads_table['R1_ID'] == reads_table['R2_ID']) &
    (reads_table['N_primers'] == 1)
)

In [181]:
reads_table

,R1_seq,R2_seq,R1_ID,R2_ID,primers,direction,N_primers,sanity
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,"(PCR5,)",R,1,True
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,"(PCR5,)",F,1,True
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,"(PCR5,)",R,1,True
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,"(PCR2,)",R,1,True
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,"(PCR6,)",F,1,True
...,...,...,...,...,...,...,...,...
995,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,"(PCR2,)",R,1,True
996,ACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTC...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCGGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:13530...,(),R,0,False
997,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,"(PCR2,)",R,1,True
998,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,"(PCR7,)",F,1,True


In [182]:
sane_reads_table = reads_table.loc[lambda df: df['sanity'] == True, :]
sane_reads_table['primer'] = sane_reads_table.apply(
    lambda r: r['primers'][0], 
    axis=1)
sane_reads_table


/opt/miniconda/envs/gatk/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,R1_seq,R2_seq,R1_ID,R2_ID,primers,direction,N_primers,sanity,primer
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,"(PCR5,)",R,1,True,PCR5
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,"(PCR5,)",F,1,True,PCR5
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,"(PCR5,)",R,1,True,PCR5
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,"(PCR2,)",R,1,True,PCR2
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,"(PCR6,)",F,1,True,PCR6
...,...,...,...,...,...,...,...,...,...
994,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:2492:...,"(PCR7,)",F,1,True,PCR7
995,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:8151:...,"(PCR2,)",R,1,True,PCR2
997,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:17454...,"(PCR2,)",R,1,True,PCR2
998,GCCAAGCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:18295...,"(PCR7,)",F,1,True,PCR7


In [183]:
counts_table=sane_reads_table.loc[:, ['R1_seq','primer', 'R1_ID']
               ].groupby(by=['R1_seq', 'primer']).count().sort_values(by=['primer','R1_ID'], ascending=[True,False])

In [184]:
counts_table

,,R1_ID
R1_seq,primer,
ATGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACC,PCR1,129
GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTCCCAAAGAGTGGGCCTCTAGCCCGGTCTGCTCTCGACCAGGGTCGTCCTCCGCTTGATAATAAGTCCATCTCTGCCGACCCAACTCATTCTGGAGCCGCCAGCGGGTGAGATCGGTGGCGGGCTCAGTTTTATAGGGTCCCCCACGACGCCGCAGACACCTGAGGGGACCAGCGAGAGGAAGCAGCCTTCCTTAGACTCCGCCC,PCR1,73
ATGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACC,PCR1,7
ATGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCAC,PCR1,6
GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTCCCAAAGAGTGGGCCTCTAGCCCGGTCTGCTCTCGACCAGGGTCGTCCTCCGCTTGATAATAAGTCCATCTCTGCCGACCCAACTCATTCTGGAGCCGCCACGCGGGTGAGATCGGTGGCGGGCTCAGTTTTATAGGGTCCCCCACGACGCCGCAGACACCTGAGGGGACCAGCGAGAGGAAGCAGCCTTCCTTAGACTCCGCCC,PCR1,6
...,...,...
GCACCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACC,PCR8,4
GCACCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACCATTCC,PCR8,3
GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTCCCAAAGAGTGGGCCTCTAAGTCCATCTCTGCCGACCCAACTCATTCTGGAGCCGCCAGCGGGTGAGATCGGTGGCGGGCTCAGTTTTATAGGGTCCCCCACGACGCCGCAGACACCTGAGGGGACCAGCGAGAGGAAGCAGCCTTCCTTAGACTCCGCCCAGGGAGGTGC,PCR8,1


In [36]:
from Bio import Align
aligner = Align.PairwiseAligner()
alignments = aligner.align(reads_table.iloc[1]['R1_seq'], reads_table.iloc[1]['R2_seq'])
alignment = alignments[0]
print(alignment)

CGTG-TC--AACTGG-GCGGAGTCT--AAGGAAGGCTGC---TTC-CTCT-C-GCTGGTCC---CCTCA--G-GTG----TCT-GC--GG-C-G-TCGT-G----GGG--GA-CC-C---T--ATAA-AACTGAG-CC--C---GCC-ACCG-AT-CTCA--CCCGCGTGGC-GGCTC-C-A-GAA----TGAG-TT-GGGTCGGCAGAGATGGACTT-A-TT--ATCAAGCGGAGGA-CGACCCTGGTCGA-GAGCAGACCGGGCTAGAGGCCCACTCTTTG-GGACTGGAC-A-C-AGTAAGTGAT-GTG-A---TTC-T--GAAA----GCAC-
-|-|-|---||-|||-||----|-|--|-|-||---|-|---|-|-||-|-|-|-||-|||---||-||--|-|||----|||-||--||-|-|-||-|-|----|||--|--||-|---|--||||-||--|---||--|---|||-|||--|--||||--|-----|||--|-|-|-|-|-|------||||-|--||-|-|||-|-|--|--||--|-||--||-|-|-||-----|--|||----|-|-||-|-|-|||--|-|||--|--||-||--|-||---||||-|-|-||-|-|-||--|---|---|||-|--||------||-|-
-G-GAT-GGAA-TGGTGC----T-TTCA-G-AA---T-CACAT-CACT-TACTG-TG-TCCAGTCC-CAAAGAGTGGGCCTCTAGCCCGGTCTGCTC-TCGACCAGGGTCG-TCCTCCGCTTGATAATAA--G--TCCATCTCTGCCGACC-CA-ACTCATTC-----TGG-AG-C-CGCCACG--CGGGTGAGAT-CGG-T-GGC-G-G--G--CT-CAGTTTTAT-A-G-GG----TC--CCC----C-ACGA-C-G-CCG--C-AGA--C--AC-CT--GAGG---GGACCAGCGAG-A-G-GA-AG--CAGCCTTCCTTA

In [37]:
basePair = {'A':'T','T':'A','G':'C','C':'G'}
basePairTrans = str.maketrans(basePair)
def rcDNA(seq):
    return seq.translate(basePairTrans) [::-1]
def cDNA(seq):
    return seq.translate(basePairTrans)

In [38]:
aligner = Align.PairwiseAligner()
alignments = aligner.align(reads_table.iloc[1]['R1_seq'], rcDNA(reads_table.iloc[1]['R2_seq']))
alignment = alignments[0]
print(alignment)

CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCAC----------
-|-|------|--|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||----------
-G-G------G--CGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACCATTCCATCC



In [44]:
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
alignments = aligner.align(reads_table.iloc[1]['R1_seq'], rcDNA(reads_table.iloc[1]['R2_seq']))
for aln in sorted(alignments):
    print("Score = %.1f:" % aln.score)
    print(aln)

Score = 240.0:
CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAG--C-A--C-----
-|-|------|--||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||--|-|--|-----
-G-G------G--CGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAGCAGACCGGGCTAGAGGCCCACTCTTTGGGACTGGACACAGTAAGTGATGTGATTCTGAAAGCACCATTCCATCC

Score = 240.0:
CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGGTCCCCTCAGGTGTCTGCGGCGTCGTGGGGGACCCTATAAAACTGAGCCCGCCACCGATCTCACCCGCGTGGCGGCTCCAGAATGAGTTGGGTCGGCAGAGATGGACTTATTATCAAGCGGAGGACGACCCTGGTCGAGAG

In [87]:
f_matches=forwardPrimerMatches(reads_table.iloc[1]['R1_seq'], primers)

In [88]:
r_matches=reversePrimerMatches(reads_table.iloc[1]['R2_seq'], primers)

In [89]:
t=tuple(p for p in f_matches if p in r_matches)

In [90]:
t

('PCR5',)

In [91]:
len(t)

1

In [92]:
primerMatches(reads_table.iloc[1]['R1_seq'], reads_table.iloc[1]['R2_seq'], primers)

('PCR5',)

In [99]:
reads_table

,R1_seq,R2_seq,R1_ID,R2_ID,sanity,primers
0,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1334:...,True,()
1,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:14841...,True,"(PCR5,)"
2,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,CGTGTCAACTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:26033...,True,()
3,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,ATGTCTCCCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:19217...,True,()
4,GCGTTCTTCTGGGCGGAGTCTAAGGAAGGCTGCTTCCTCTCGCTGG...,GGATGGAATGGTGCTTTCAGAATCACATCACTTACTGTGTCCAGTC...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,GWNJ-1012:437:GW201115869th.Miseq:1:2101:1542:...,True,"(PCR6,)"
